# Processo de Seleção de Variáveis Usando P-Value.

### Importando libs e funções:

Importando libs

In [0]:
import pandas as pd
import random
import numpy as np
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

Importando funções

In [0]:
# Função de escalonamento
def feature_scaling(data):
    sc = StandardScaler()
    return sc.fit_transform(data)

### Etapa de exploração e tratamento dos **dados**

Importando o dataset do nosso estudo. O objetivo do modelo de regressão será de predizer o preço das casas de acordo com diferentes atributos como: localização, área do lote, garagem, etc.

Fonte: [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/r4phael/ml-course/master/data/pricing_houses.csv')

#Selecionando algumas features dos dados para uma melhor visualização do problema
df = df.loc[:, ['LotArea', 'PoolArea', 'GarageArea', 'OverallCond','YearBuilt', 'YrSold', 'Fireplaces',
                'SalePrice']]

Descrevendo o dataset

In [4]:
df.describe()

,LotArea,PoolArea,GarageArea,OverallCond,YearBuilt,YrSold,Fireplaces,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,2.758904,472.980137,5.575342,1971.267808,2007.815753,0.613014,180921.195890
std,9981.264932,40.177307,213.804841,1.112799,30.202904,1.328095,0.644666,79442.502883
min,1300.000000,0.000000,0.000000,1.000000,1872.000000,2006.000000,0.000000,34900.000000
25%,7553.500000,0.000000,334.500000,5.000000,1954.000000,2007.000000,0.000000,129975.000000
50%,9478.500000,0.000000,480.000000,5.000000,1973.000000,2008.000000,1.000000,163000.000000
75%,11601.500000,0.000000,576.000000,6.000000,2000.000000,2009.000000,1.000000,214000.000000
max,215245.000000,738.000000,1418.000000,9.000000,2010.000000,2010.000000,3.000000,755000.000000


Visualizando o dataset

In [5]:
df.head(5)

,LotArea,PoolArea,GarageArea,OverallCond,YearBuilt,YrSold,Fireplaces,SalePrice
0,8450,0,548,5,2003,2008,0,208500
1,9600,0,460,8,1976,2007,1,181500
2,11250,0,608,5,2001,2008,1,223500
3,9550,0,642,5,1915,2006,1,140000
4,14260,0,836,5,2000,2008,1,250000


Preenchendo os valores númericos nulos (NA) com a mediana.

In [6]:
df = df.fillna(df.median())

df.head(5)

,LotArea,PoolArea,GarageArea,OverallCond,YearBuilt,YrSold,Fireplaces,SalePrice
0,8450,0,548,5,2003,2008,0,208500
1,9600,0,460,8,1976,2007,1,181500
2,11250,0,608,5,2001,2008,1,223500
3,9550,0,642,5,1915,2006,1,140000
4,14260,0,836,5,2000,2008,1,250000


## Forward Elimination


### Etapa de Seleção e Tratamento dos Dados

Definindo as variáveis indepedentes e dependentes, normalição das features e divisão do dataset em conjunto de treinamento e testes:

In [0]:
X = df[df.columns[~df.columns.isin(['SalePrice'])]].values
y = df['SalePrice'].values.reshape(-1,1)

# Normalização das features:
X = feature_scaling(X)

### Realizando o Processo de Foward Elimination


Realizando o processo de Forward Elimination. Esse processo é realizado através de uma análise incremental da contribuição das features ao modelo final. Portanto, a cada iteração é adicionada uma feature que deverá ser analisada seu impacto no modelo através do *p-value*.

Primeiro, será inserido uma coluna preenchida com valores 1 no começo da matriz de feature para que seja realizado os cálculos necessários. 

In [0]:
X = np.append(arr = np.ones((1460,1)).astype(int), values = X, axis =1)

Divisão do dataset em conjunto de treinamento e testes:

In [9]:
# Dividindo os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train[1:5,:5]

array([[ 1.        , -0.26857781, -0.06869175, -0.43503222,  1.28068524],
       [ 1.        , -0.1743691 , -0.06869175, -2.21296298,  1.28068524],
       [ 1.        , -0.33241925, -0.06869175, -1.09005935,  1.28068524],
       [ 1.        , -0.55290771, -0.06869175, -0.77190332,  0.38174271]])

Adição da 1ª Feature (*LotArea* - Área do Lote):

In [10]:
# Importando a package.
import statsmodels.regression.linear_model as sm

X_opt = X_train[:, [0,1]]
regressor_ols = sm.OLS(endog = y_train, exog = X_train[:, [0,1]]).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     88.93
Date:                Sat, 30 Nov 2019   Prob (F-statistic):           2.13e-20
Time:                        13:40:20   Log-Likelihood:                -14760.
No. Observations:                1168   AIC:                         2.952e+04
Df Residuals:                    1166   BIC:                         2.953e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.811e+05   2180.395     83.063      0.000    1.77e+05    1.85e+05
x1          1.907e+04   2022.624      9.430      0.000    1.51e+04     2.3e+04
==============================================================================
Omnibus:                      425.777   Durbin-Watson:                   2.049
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1973.518
Skew:                           1.657   Prob(JB):                         0.00
Kurtosis:                       8.437   Cond. No.                         1.08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Analisando os valores acima, vimos que as features X1(*LotArea* - Área do lote) possui um P-value significativo, ou seja, dentro do intervalo definido (SL = .05) .Portanto, deixamos a mesma e escolhemos outra feature para incrementar no modelo conforme o processo de Forward Elimination.  

**Obs: Definimos um level de significância (SL) de .05 para que as features permaneçam no modelo (SL = .05).**

Calculando os coeficientes com a adição da 2ª feature (*PoolArea* - Área da Piscina):


In [0]:
# Selecionando apenas as features de indice 0-const, 1-LotArea, 2-PoolArea
X_opt = X_train[:, [0,1,2]]
regressor_ols = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     50.31
Date:                Fri, 29 Nov 2019   Prob (F-statistic):           1.10e-21
Time:                        14:05:29   Log-Likelihood:                -14754.
No. Observations:                1168   AIC:                         2.951e+04
Df Residuals:                    1165   BIC:                         2.953e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.811e+05   2171.175     83.405      0.000    1.77e+05    1.85e+05
x1           1.85e+04   2021.631      9.149      0.000    1.45e+04    2.25e+04
x2          6952.4530   2102.250      3.307      0.001    2827.833    1.11e+04
==============================================================================
Omnibus:                      375.808   Durbin-Watson:                   2.045
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1345.385
Skew:                           1.540   Prob(JB):                    7.13e-293
Kurtosis:                       7.261   Cond. No.                         1.11
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Analisando os valores acima, vimos que as features X1 e X2 possuem um P-value significativo, ou seja, dentro do intervalo definido (SL = .05) .Portanto, deixamos elas e escolhemos outra feature para incrementar no modelo conforme o processo de Forward Elimination. 

Calculando os coeficientes com a adição da 3ª feature (*GarageArea* - Área da Garagem):


In [0]:
# Selecionando apenas as features de indice 0-const, 1-LotArea, 2-PoolArea, 3-GarageArea
X_opt = X_train[:, [0,1,2,3]]
regressor_ols = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.417
Model:                            OLS   Adj. R-squared:                  0.416
Method:                 Least Squares   F-statistic:                     277.7
Date:                Fri, 29 Nov 2019   Prob (F-statistic):          6.26e-136
Time:                        14:05:29   Log-Likelihood:                -14487.
No. Observations:                1168   AIC:                         2.898e+04
Df Residuals:                    1164   BIC:                         2.900e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.805e+05   1728.549    104.432      0.000    1.77e+05    1.84e+05
x1          1.114e+04   1634.120      6.815      0.000    7930.811    1.43e+04
x2          4089.5066   1677.167      2.438      0.015     798.898    7380.116
x3           4.63e+04   1783.141     25.968      0.000    4.28e+04    4.98e+04
==============================================================================
Omnibus:                      326.351   Durbin-Watson:                   2.019
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2200.071
Skew:                           1.113   Prob(JB):                         0.00
Kurtosis:                       9.345   Cond. No.                         1.25
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Analisando os valores acima, vimos que todas as features possuem um P-value significativo, ou seja, dentro do intervalo definido (SL = .05) .Portanto, deixamos elas e escolhemos outra feature para incrementar no modelo conforme o processo de Forward Elimination. 

Calculando os coeficientes com a adição da 4ª feature (*OverallCond* - Condição Geral):


In [0]:
# Selecionando apenas as features de indice 0-const, 1-LotArea, 2-PoolArea, 3-GarageArea, 4-OverallCond
X_opt = X_train[:, [0,1,2,3,4]]
regressor_ols = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.417
Model:                            OLS   Adj. R-squared:                  0.415
Method:                 Least Squares   F-statistic:                     208.2
Date:                Fri, 29 Nov 2019   Prob (F-statistic):          1.06e-134
Time:                        14:05:29   Log-Likelihood:                -14487.
No. Observations:                1168   AIC:                         2.898e+04
Df Residuals:                    1163   BIC:                         2.901e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.805e+05   1729.280    104.384      0.000    1.77e+05    1.84e+05
x1          1.112e+04   1635.325      6.800      0.000    7911.844    1.43e+04
x2          4088.4437   1677.789      2.437      0.015     796.611    7380.276
x3           4.64e+04   1802.016     25.749      0.000    4.29e+04    4.99e+04
x4           652.2132   1741.565      0.374      0.708   -2764.748    4069.175
==============================================================================
Omnibus:                      326.932   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2206.916
Skew:                           1.114   Prob(JB):                         0.00
Kurtosis:                       9.354   Cond. No.                         1.31
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Analisando os valores acima, vimos que toda a feature *OverallCond* (Condição Geral da Casa) não possui um P-value significativo, ou seja, dentro do intervalo definido (SL = .05), pois a mesma possui um valor de 0.708. Portanto, deixamos elas e escolhemos outra feature para incrementar no modelo conforme o processo de Forward Elimination. 

**Final:** Analisando os valores acima, vimos que todas as features possuem um P-value significativo, ou seja, dentro do intervalo definido (SL = .05) .Portanto, não iremos adicionar ela ao modelo final, visto que ela não impacta de maneira positiva ao modelo. Finalmente, esse ciclo se repete até que todas as features sejam analisadas.

Treinando o modelo com o conjunto de treinamento.

**Obs:** A feature de índice 0 é uma constante (1) que foi somente criada para análise do processo de seleção. Portanto, não foi inserida no treinamento do modelo devido. 

In [0]:
regressor = LinearRegression()
regressor.fit(X_test[:, [1,2,3]], y_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Analisando o novo score do modelo com a métrica r2

In [0]:
regressor.score(X_test[:, [1,2,3]], y_test)

0.41858756810845454

##  Backward Elimination

### Etapa de Seleção e Tratamento dos Dados

Definindo as variáveis indepedentes e dependentes, normalição das features e divisão do dataset em conjunto de treinamento e testes:

In [0]:
X = df[df.columns[~df.columns.isin(['SalePrice'])]].values
y = df['SalePrice'].values.reshape(-1,1)

# Normalização das features:
X = feature_scaling(X)

### Realizando o Processo de Backward Elimination


Realizando o processo de Backward Elimination. Esse processo é realizado através de uma análise da contribuição de todas as features ao modelo final. Portanto, a cada iteração é removida uma feature que deverá ser analisada seu impacto no modelo através do *p-value*.

Primeiro, será inserido uma coluna preenchida com valores 1 no começo da matriz de feature para que seja realizado os cálculos necessários. 

In [0]:
X = np.append(arr = np.ones((1460,1)).astype(int), values = X, axis =1)

Divisão do dataset em conjunto de treinamento e testes:

In [13]:
# Dividindo os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train[1:5,:5]

array([[ 1.        , -0.26857781, -0.06869175, -0.43503222,  1.28068524],
       [ 1.        , -0.1743691 , -0.06869175, -2.21296298,  1.28068524],
       [ 1.        , -0.33241925, -0.06869175, -1.09005935,  1.28068524],
       [ 1.        , -0.55290771, -0.06869175, -0.77190332,  0.38174271]])

Analisando todas as features no modelo:

In [14]:
# Importando a package.
import statsmodels.regression.linear_model as sm

X_opt = X_train[:, [0,1,2,3,4,5,6,7]]
regressor_ols = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.572
Model:                            OLS   Adj. R-squared:                  0.570
Method:                 Least Squares   F-statistic:                     221.7
Date:                Sat, 30 Nov 2019   Prob (F-statistic):          7.88e-209
Time:                        13:42:39   Log-Likelihood:                -14307.
No. Observations:                1168   AIC:                         2.863e+04
Df Residuals:                    1160   BIC:                         2.867e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.808e+05   1483.694    121.883      0.000    1.78e+05    1.84e+05
x1          8061.6719   1446.960      5.571      0.000    5222.720    1.09e+04
x2          3336.4581   1448.058      2.304      0.021     495.353    6177.563
x3          3.006e+04   1765.796     17.021      0.000    2.66e+04    3.35e+04
x4          8726.2152   1607.366      5.429      0.000    5572.546    1.19e+04
x5          2.612e+04   1783.040     14.647      0.000    2.26e+04    2.96e+04
x6           236.3377   1494.222      0.158      0.874   -2695.342    3168.017
x7          2.165e+04   1584.915     13.663      0.000    1.85e+04    2.48e+04
==============================================================================
Omnibus:                      464.282   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5052.077
Skew:                           1.521   Prob(JB):                         0.00
Kurtosis:                      12.724   Cond. No.                         2.04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Analisando os valores acima, vimos que a feature X6 (*YrSold* - Ano de Venda) possui p-value de .874, enquanto que as outras features possui um valor abaixo do limiar (SL = .05). Portanto, tal feature deve ser retirada seguindo o processo de Backward Selection.

**Obs: Definimos um level de significância de .05 para que as features permaneçam no modelo (SL = .05).**

Remoção da  6ª Feature (*YrSold* - Ano de Venda):

In [0]:
#Analisando todas as features, exceto 6-YrSold

X_opt = X_train[:, [0,1,2,3,4,5,7]]
regressor_ols = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.572
Model:                            OLS   Adj. R-squared:                  0.570
Method:                 Least Squares   F-statistic:                     258.8
Date:                Fri, 29 Nov 2019   Prob (F-statistic):          4.75e-210
Time:                        14:05:29   Log-Likelihood:                -14307.
No. Observations:                1168   AIC:                         2.863e+04
Df Residuals:                    1161   BIC:                         2.866e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.808e+05   1483.065    121.935      0.000    1.78e+05    1.84e+05
x1          8062.8559   1446.333      5.575      0.000    5225.137    1.09e+04
x2          3320.0150   1443.714      2.300      0.022     487.434    6152.596
x3          3.005e+04   1764.767     17.028      0.000    2.66e+04    3.35e+04
x4          8732.9953   1606.119      5.437      0.000    5581.775    1.19e+04
x5          2.612e+04   1781.992     14.658      0.000    2.26e+04    2.96e+04
x6          2.165e+04   1584.243     13.668      0.000    1.85e+04    2.48e+04
==============================================================================
Omnibus:                      464.454   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5049.657
Skew:                           1.522   Prob(JB):                         0.00
Kurtosis:                      12.721   Cond. No.                         2.04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Analisando os valores acima, vimos que todas as features possuem um valor muito próximo de zero, exceto a feature X6 (*PoolArea* - Área da Piscina) que possui p-value de .022. Apesar da feature PoolArea se diferenciar das demais, ela ainda está abaixo do limiar definido (SL = .05). 

**Obs: Definimos um level de significância de .05 para que as features permaneçam no modelo (SL = .05).**

**Final:** Portanto, seguindo o processo de Backward Selection, todas as features acima devem ser mantidas no model.

Treinamento do modelo com todas as features que tem um SL abaixo de .05, 

**Obs:** A feature de índice 0 é uma constante (1) que foi somente criada para análise do processo de seleção. Portanto, não foi inserida no treinamento do modelo devido. 

In [0]:
regressor = LinearRegression()
#Treinando o modelo de Regressão com todas as features, exceto X6
regressor.fit(X_train[:, [1,2,3,4,5,7]], y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Analisando o novo score do modelo com a métrica r²:

In [0]:
regressor.score(X_test[:, [1,2,3,4,5,7]], y_test)

0.5633529452543352